In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [2]:
import json
import os

x = json.load(open("train_sample_videos/metadata.json"))
real = [file for file in x if x[file]["label"] == "REAL"]
fake = [file for file in x if x[file]["label"] == "FAKE"]

In [2]:
df = pd.read_csv('features.csv')
df.head()

,FileName,rmse,zcr,tempo,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,spectral_rolloff,mfcc1,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,bbvgxeczei.mp4,0.008028,0.038316,114.843750,3044.967492,5054.150023,16.414213,0.005748,5682.982621,-490.07425,...,4.405778,-4.854592,-7.056202,-2.636595,-9.831662,-7.514642,-8.755273,-1.896833,-2.603562,FAKE
1,agqphdxmwt.mp4,0.005214,0.041688,123.046875,3522.410712,5273.395038,15.003604,0.007969,7564.151674,-544.73663,...,12.557711,-6.926794,-1.758291,-2.297427,-6.438809,-0.577922,-3.541346,-1.711840,-0.793243,FAKE
2,cglxirfaey.mp4,0.000189,0.073073,120.185320,5057.993566,5686.422947,13.394466,0.038380,12171.683589,-809.70215,...,7.754112,-0.916289,3.209106,-1.237075,1.662170,-4.138675,3.358046,-2.736235,0.954534,FAKE
3,ekhacizpah.mp4,0.032205,0.023700,126.048018,2369.255016,4806.154161,16.368527,0.001560,3229.456789,-340.31854,...,2.123348,-3.004771,-2.435924,5.287929,-9.668341,0.485538,-0.291376,6.767036,-2.478494,FAKE
4,ahbweevwpv.mp4,0.031066,0.019815,117.453835,2084.400204,4570.329358,15.247853,0.001029,2592.238562,-361.49948,...,5.199107,3.196825,-3.196581,0.345530,-10.903251,2.098206,-4.634393,4.790202,-2.696295,FAKE


In [11]:
df["label"].value_counts()

label
FAKE    323
REAL     77
Name: count, dtype: int64

In [4]:
df.drop('FileName', axis=1, inplace=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.2, random_state=42)

SVC

In [9]:
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [10]:
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        FAKE       0.81      1.00      0.90        65
        REAL       0.00      0.00      0.00        15

    accuracy                           0.81        80
   macro avg       0.41      0.50      0.45        80
weighted avg       0.66      0.81      0.73        80



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [13]:
y_pred = rfc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        FAKE       0.81      0.91      0.86        65
        REAL       0.14      0.07      0.09        15

    accuracy                           0.75        80
   macro avg       0.48      0.49      0.47        80
weighted avg       0.68      0.75      0.71        80

